In [ ]:
import pyspark
from pyspark.sql import SparkSession
import json
import random
import pandas as pd
import pymongo
import re
from itertools import tee

In [ ]:
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
spark = SparkSession.builder.getOrCreate()
client.drop_database('AviFest')

In [ ]:
db = client['AviFest']

In [ ]:
df = spark.read.json('persons.json').toPandas()
def preprocessing(row):
    row['sexe'] = row['sexe'][0]
    firstname, lastname, *_ = row['name'].split(' ')
    row['firstname'] = firstname
    row['lastname'] = lastname
    
    del row['name']
    
    return row


df = df.apply(preprocessing, axis=1)

In [ ]:
collection = db['users']

for index, row in df.iterrows():
    collection.insert_one({
        'phone': row['phone'],
        'email': row['email'],
        'sexe': row['sexe'],
        'age': row['age'],
        'firstname': row['firstname'],
        'lastname': row['lastname'],
    })

In [ ]:
for document in collection.find({}):
    print(document)

In [ ]:
df = spark.read.options(header='True', inferSchema='True', delimiter=',') \
  .csv("show.csv")
df = df.drop(*filter(lambda x: '_c' in x, df.columns)).toPandas()

In [ ]:
collection = db['shows']

for index, row in df.iterrows():
    collection.insert_one({
        'title': row['title'],
        'type': row['type'],
        'artist': row['artist'],
        'price': '',
        'nbPlace': row['nb_place'],
        'idLocation': '',
    })

In [ ]:
for document in collection.find({}):
    print(document)

In [ ]:
collection = db['parkings']

rows, get_nb_rows = tee(spark.read.option("multiline", "true").json('parkings.json').toPandas().iterrows())
nb_parkings = len(list((get_nb_rows)))

list_pmr = list(map(lambda x: x < 0.75, [random.random() for _ in range(nb_parkings)]))

description: str
for (_, (((lat, long), _), (description, name), _)), pmr, bus in zip(rows, list_pmr, reversed(list_pmr)):
    
    try:
        slot = int(re.search(r'\*\*(\d+)\*\*', description).group(1))
        
        if not slot:
            slot = 'unknown'
    except: slot = 'unknown'
    
    collection.insert_one({
        'type': 'voiture',
        'name': name,
        'nbslots': slot,
        'paying': not 'gratuit' in description.lower(),
        'busFestiv': bus,
        'pmr': pmr,
        'idLocation': '',
    })

In [ ]:
for document in collection.find({}):
    print(document)